## **Kuis 3 - Enkripsi Algoritma DES**
* Amanda Putri Aprilliani (105222001)
* Ni Putu Merta Bhuana Ningsih (105222008)
* Raihan Akira Rahmaputra (105222040)     


-----------------------------

> **Import Library dan Setup**

In [1]:
import numpy as np

print("=== IMPLEMENTASI ALGORITMA DES (DATA ENCRYPTION STANDARD) ===")
print("Program ini mendukung input kunci custom 64-bit binary dari pengguna")
print("=" * 70)

=== IMPLEMENTASI ALGORITMA DES (DATA ENCRYPTION STANDARD) ===
Program ini mendukung input kunci custom 64-bit binary dari pengguna


> **Konstanta dan Matriks DES**

In [2]:
# Matriks permutasi awal (Initial Permutation - IP)
IP = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

# Matriks permutasi awal balikan (IP^-1)
IP_INV = [
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41, 9, 49, 17, 57, 25
]

# Matriks permutasi kompresi PC-1 untuk kunci
PC1 = [
    57, 49, 41, 33, 25, 17, 9, 1,
    58, 50, 42, 34, 26, 18, 10, 2,
    59, 51, 43, 35, 27, 19, 11, 3,
    60, 52, 44, 36, 63, 55, 47, 39,
    31, 23, 15, 7, 62, 54, 46, 38,
    30, 22, 14, 6, 61, 53, 45, 37,
    29, 21, 13, 5, 28, 20, 12, 4
]

# Matriks permutasi kompresi PC-2 untuk kunci internal
PC2 = [
    14, 17, 11, 24, 1, 5, 3, 28,
    15, 6, 21, 10, 23, 19, 12, 4,
    26, 8, 16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55, 30, 40,
    51, 45, 33, 48, 44, 49, 39, 56,
    34, 53, 46, 42, 50, 36, 29, 32
]

# Jumlah pergeseran bit untuk setiap putaran
SHIFT_AMOUNTS = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]


> **Matriks Ekspansi dan S-box**

In [3]:
# Matriks ekspansi E (32 bit -> 48 bit)
E = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

# Matriks permutasi P
P = [
    16, 7, 20, 21, 29, 12, 28, 17,
    1, 15, 23, 26, 5, 18, 31, 10,
    2, 8, 24, 14, 32, 27, 3, 9,
    19, 13, 30, 6, 22, 11, 4, 25
]

# S-Box (Substitution Box) - 8 buah kotak substitusi
S_BOX = [
    # S1
    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],
    
    # S2
    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],
    
    # S3
    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],
    
    # S4
    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],
    
    # S5
    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],
    
    # S6
    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],
    
    # S7
    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],
    
    # S8
    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
]

> **Fungsi yang dibutuhkan dalam DES**

In [4]:
# Fungsi untuk validasi input binary 64-bit
def validate_binary(s, name):
    """Validasi input binary 64-bit"""
    if len(s) != 64:
        raise ValueError(f"{name} harus tepat 64 bit, diterima {len(s)} bit")
    if not all(c in '01' for c in s):
        raise ValueError(f"{name} harus berisi hanya 0 dan 1")
    return s

# Fungsi untuk mengkonversi string ke array bit
def string_to_bits(text):
    """Mengkonversi string ke array bit"""
    bits = []
    for char in text:
        # Konversi karakter ke ASCII, lalu ke 8-bit binary
        ascii_val = ord(char)
        char_bits = [(ascii_val >> i) & 1 for i in range(7, -1, -1)]
        bits.extend(char_bits)
    return bits

# Fungsi untuk mengkonversi array bit ke string
def bits_to_string(bits):
    """Mengkonversi array bit ke string"""
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) == 8:  # Pastikan byte lengkap
            ascii_val = 0
            for j, bit in enumerate(byte):
                ascii_val += bit * (2 ** (7-j))
            chars.append(chr(ascii_val))
    return ''.join(chars)

# Fungsi permutasi data
def permute(data, permutation_table):
    """Melakukan permutasi data berdasarkan tabel permutasi"""
    return [data[i-1] for i in permutation_table]

# Fungsi untuk menggeser bit ke kiri
def left_shift(bits, n):
    """Melakukan left shift pada array bit sebanyak n posisi"""
    return bits[n:] + bits[:n]

# Fungsi XOR untuk dua array bit
def xor(bits1, bits2):
    """Operasi XOR antara dua array bit"""
    return [b1 ^ b2 for b1, b2 in zip(bits1, bits2)]

# Fungsi mengkonversi array bit ke representasi hexadecimal
def bits_to_hex(bits):
    """Mengkonversi array bit ke representasi hexadecimal"""
    hex_str = ""
    for i in range(0, len(bits), 4):
        nibble = bits[i:i+4]
        while len(nibble) < 4:
            nibble.append(0)
        hex_val = nibble[0]*8 + nibble[1]*4 + nibble[2]*2 + nibble[3]*1
        hex_str += format(hex_val, 'X')
    return hex_str

# Fungsi untuk memformat bit
def format_bits(bits, group_size=8):
    """Format bit untuk tampilan yang lebih rapi"""
    formatted = ""
    for i in range(0, len(bits), group_size):
        group = bits[i:i+group_size]
        formatted += ''.join(map(str, group)) + " "
    return formatted.strip()

# Fungsi untuk mengkonversi string binary ke array bit
def binary_string_to_bits(binary_str):
    """Konversi string binary ke array bit"""
    return [int(b) for b in binary_str]

# Fungsi untuk mengkonversi array bit ke string binary
def bits_to_binary_string(bits):
    """Konversi array bit ke string binary"""
    return ''.join(map(str, bits))

> **Pembangkitan Kunci Internal**

In [5]:
def generate_keys(key_bits):
    """Membangkitkan 16 kunci internal dari kunci eksternal 64-bit"""
    print("\n--- PROSES PEMBANGKITAN KUNCI INTERNAL ---")
    print(f"Kunci eksternal (64-bit): {format_bits(key_bits)}")
    
    # Langkah 1: Permutasi PC-1 (64-bit -> 56-bit)
    key_56bit = permute(key_bits, PC1)
    print(f"Setelah PC-1 (56-bit): {format_bits(key_56bit, 7)}")
    
    # Langkah 2: Bagi menjadi C0 dan D0 (masing-masing 28-bit)
    C = key_56bit[:28]
    D = key_56bit[28:]
    
    print(f"C0 (28-bit): {format_bits(C, 7)}")
    print(f"D0 (28-bit): {format_bits(D, 7)}")
    
    keys = []
    
    # Generate 16 kunci internal
    for round_num in range(16):
        # Left shift sesuai tabel
        shift_amount = SHIFT_AMOUNTS[round_num]
        C = left_shift(C, shift_amount)
        D = left_shift(D, shift_amount)
        
        # Gabungkan C dan D, lalu permutasi PC-2
        CD = C + D
        round_key = permute(CD, PC2)
        keys.append(round_key)
        
        print(f"K{round_num+1:2d} (48-bit): {format_bits(round_key, 6)}")
    
    return keys

> **Fungsi f (Transformasi)**

In [6]:
def function_f(R, key):
    """Fungsi f dalam jaringan Feistel"""
    # Langkah 1: Ekspansi R dari 32-bit ke 48-bit
    expanded_R = permute(R, E)
    
    # Langkah 2: XOR dengan kunci
    xor_result = xor(expanded_R, key)
    
    # Langkah 3: Substitusi menggunakan S-Box
    substituted = []
    for i in range(8):  # 8 S-Box
        # Ambil 6-bit untuk S-Box ke-i
        six_bits = xor_result[i*6:(i+1)*6]
        
        # Bit pertama dan terakhir menentukan baris
        row = six_bits[0] * 2 + six_bits[5]
        
        # 4 bit tengah menentukan kolom
        col = six_bits[1] * 8 + six_bits[2] * 4 + six_bits[3] * 2 + six_bits[4]
        
        # Ambil nilai dari S-Box dan konversi ke 4-bit
        sbox_val = S_BOX[i][row][col]
        four_bits = [(sbox_val >> j) & 1 for j in range(3, -1, -1)]
        substituted.extend(four_bits)
    
    # Langkah 4: Permutasi P
    result = permute(substituted, P)
    return result

> **Algoritma DES Utama**

In [7]:
def des_encrypt_block(plaintext_64bit, keys):
    """Enkripsi satu blok 64-bit menggunakan DES"""
    print(f"\n--- ENKRIPSI BLOK ---")
    print(f"Plainteks (64-bit): {format_bits(plaintext_64bit)}")
    
    # Langkah 1: Permutasi awal (IP)
    permuted = permute(plaintext_64bit, IP)
    print(f"Setelah IP: {format_bits(permuted)}")
    
    # Langkah 2: Bagi menjadi L0 dan R0
    L = permuted[:32]
    R = permuted[32:]
    
    print(f"L0: {format_bits(L, 8)}")
    print(f"R0: {format_bits(R, 8)}")
    
    # Langkah 3: 16 putaran Feistel
    for round_num in range(16):
        print(f"\n--- PUTARAN {round_num+1} ---")
        
        # Simpan R sebelumnya
        prev_R = R[:]
        
        # Hitung fungsi f
        f_result = function_f(R, keys[round_num])
        print(f"f(R{round_num}, K{round_num+1}): {format_bits(f_result, 8)}")
        
        # R_new = L XOR f(R, K)
        R = xor(L, f_result)
        
        # L_new = R_previous
        L = prev_R
        
        print(f"L{round_num+1}: {format_bits(L, 8)}")
        print(f"R{round_num+1}: {format_bits(R, 8)}")
    
    # Langkah 4: Gabungkan R16 dan L16 (perhatikan urutan!)
    pre_ciphertext = R + L
    print(f"\nPra-ciphertext (R16L16): {format_bits(pre_ciphertext)}")
    
    # Langkah 5: Permutasi awal balikan (IP^-1)
    ciphertext = permute(pre_ciphertext, IP_INV)
    print(f"Ciphertext (64-bit): {format_bits(ciphertext)}")
    
    return ciphertext

def des_encrypt(plaintext, key_binary):
    """Enkripsi teks menggunakan DES dengan kunci binary"""
    print("="*70)
    print("MEMULAI PROSES ENKRIPSI DES")
    print("="*70)
    
    # Validasi dan konversi kunci binary ke bit array
    key_binary = validate_binary(key_binary, "Kunci")
    key_bits = binary_string_to_bits(key_binary)
    
    # Generate kunci internal
    keys = generate_keys(key_bits)
    
    # Konversi plaintext ke bit
    plaintext_bits = string_to_bits(plaintext)
    
    # Padding jika perlu (untuk kelipatan 64-bit)
    while len(plaintext_bits) % 64 != 0:
        plaintext_bits.append(0)
    
    print(f"\nPlaintext: '{plaintext}'")
    print(f"Plaintext (bit): {format_bits(plaintext_bits)}")
    print(f"Panjang: {len(plaintext_bits)} bit ({len(plaintext_bits)//64} blok)")
    
    # Enkripsi setiap blok 64-bit
    ciphertext_bits = []
    for i in range(0, len(plaintext_bits), 64):
        block = plaintext_bits[i:i+64]
        encrypted_block = des_encrypt_block(block, keys)
        ciphertext_bits.extend(encrypted_block)
    
    print(f"\n" + "="*70)
    print("HASIL ENKRIPSI")
    print("="*70)
    print(f"Ciphertext (bit): {format_bits(ciphertext_bits)}")
    print(f"Ciphertext (hex): {bits_to_hex(ciphertext_bits)}")
    print(f"Ciphertext (binary): {bits_to_binary_string(ciphertext_bits)}")
    
    return ciphertext_bits

print("[OK] Algoritma DES berhasil dimuat")

[OK] Algoritma DES berhasil dimuat


> **Program Utama DES**

In [8]:
print("\n" + "="*70)
print("PROGRAM ENKRIPSI DES")
print("="*70)
print("Program ini mendukung input kunci custom dari pengguna")
print("="*70)

# Input dari pengguna
try:
    print("\n--- INPUT DATA ---")
    plaintext = input("Masukkan teks yang akan dienkripsi: ")
    
    print("\nPilih mode kunci:")
    print("1. Gunakan kunci default")
    print("2. Masukkan kunci custom (64-bit binary)")
    
    choice = input("Pilihan (1/2): ").strip()
    
    if choice == "1":
        # Kunci default yang sudah ditentukan
        key_binary = "1010010010000010100111001000111010001110100000101000111010011100"
        print(f"Menggunakan kunci default: {key_binary}")
    elif choice == "2":
        # Input kunci custom dari user
        key_binary = input("Masukkan kunci 64-bit binary (contoh: 1010...): ").strip()
        # Validasi akan dilakukan di fungsi des_encrypt
    else:
        print("Pilihan tidak valid, menggunakan kunci default")
        key_binary = "1010010010000010100111001000111010001110100000101000111010011100"
    
    # Jalankan enkripsi
    ciphertext_bits = des_encrypt(plaintext, key_binary)
    
    print(f"\n[OK] Enkripsi berhasil!")
    print(f"Teks asli: '{plaintext}'")
    print(f"Kunci yang digunakan: {key_binary}")
    print(f"Hasil enkripsi (binary): {bits_to_binary_string(ciphertext_bits)}")
    print(f"Hasil enkripsi (hex): {bits_to_hex(ciphertext_bits)}")
    
except ValueError as e:
    print(f"\n[ERROR] Error: {e}")
    print("Pastikan kunci yang dimasukkan adalah 64-bit binary (hanya 0 dan 1)")
except Exception as e:
    print(f"\n[ERROR] Terjadi kesalahan: {e}")

print("\n" + "="*70)


PROGRAM ENKRIPSI DES
Program ini mendukung input kunci custom dari pengguna

--- INPUT DATA ---

Pilih mode kunci:
1. Gunakan kunci default
2. Masukkan kunci custom (64-bit binary)
Menggunakan kunci default: 1010010010000010100111001000111010001110100000101000111010011100
MEMULAI PROSES ENKRIPSI DES

--- PROSES PEMBANGKITAN KUNCI INTERNAL ---
Kunci eksternal (64-bit): 10100100 10000010 10011100 10001110 10001110 10000010 10001110 10011100
Setelah PC-1 (56-bit): 1111111 1000000 0000000 0011000 0111101 0110111 0111011 1000100
C0 (28-bit): 1111111 1000000 0000000 0011000
D0 (28-bit): 0111101 0110111 0111011 1000100
K 1 (48-bit): 000111 110100 100100 010001 101110 110101 111011 000111
K 2 (48-bit): 000011 110100 100100 001001 011010 110011 101100 110110
K 3 (48-bit): 000010 110010 000110 001101 101101 010110 100110 111110
K 4 (48-bit): 000110 010000 110010 001001 011001 010001 101011 010111
K 5 (48-bit): 010100 010010 100010 101000 110101 111010 000011 111111
K 6 (48-bit): 100100 001010 1